# Tarea 1 - Taller de Deep Learning

**Johny Kidd: 228175**  



# Imports


In [ ]:
# Core Python
import os
import random
import numpy as np
from collections import Counter


# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary


# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Reproducibilidad
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando device:", device)

# Weights & Biases
import wandb
wandb.login() 


## Seteo de Variables

In [ ]:
BATCH_SIZE = 128
NUM_WORKERS = 4

# 1. Carga de Dataset


In [ ]:
transform = transforms.Compose([
    transforms.Resize((160, 160)), 
    transforms.ToTensor()
])

# Cargamos Imagenette 160px
train_dataset = datasets.Imagenette(
    root="./data", size="160px", split="train", download=False, transform=transform
)
val_dataset = datasets.Imagenette(
    root="./data", size="160px", split="val", download=False, transform=transform
)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"Train size: {len(train_dataset)}")
print(f"Val size: {len(val_dataset)}")
print(f"Clases: {train_dataset.classes}")


In [ ]:
# Mostrar algunos ejemplos de imágenes
def show_samples(dataset, n=8):
    indices = np.random.choice(len(dataset), n, replace=False)  # Selecciona índices aleatorios
    fig, axes = plt.subplots(1, n, figsize=(15, 3))
    for i, idx in enumerate(indices):
        img, label = dataset[idx]
        axes[i].imshow(np.transpose(img.numpy(), (1, 2, 0)))
        axes[i].set_title(dataset.classes[label], fontsize=8)
        axes[i].axis("off")
    plt.show()

show_samples(train_dataset)

# 3. Análisis Exploratorio de Datos (EDA)

## Balanceo de Clases

In [ ]:
class_names = train_dataset.classes
class_names_simple = [c[0] if isinstance(c, tuple) else c for c in class_names]

train_labels = [label for _, label in train_dataset._samples]
val_labels = [label for _, label in val_dataset._samples]

train_counts = Counter(train_labels)
val_counts = Counter(val_labels)

train_distribution = [train_counts[i] for i in range(len(class_names))]
val_distribution = [val_counts[i] for i in range(len(class_names))]

plt.figure(figsize=(8,4))
sns.barplot(x=class_names_simple, y=train_distribution, color="skyblue", label="Train")
sns.barplot(x=class_names_simple, y=val_distribution, color="salmon", alpha=0.7, label="Val")
plt.xticks(rotation=45)
plt.title("Distribución de imágenes por clase")
plt.ylabel("Cantidad de imágenes")
plt.legend()
plt.show()


## Tamaño y Aspect Ratio

In [ ]:

widths, heights, aspect_ratios = [], [], []


for img, _ in train_dataset:
    pil_img = transforms.ToPILImage()(img)
    w, h = pil_img.size  
    widths.append(w)
    heights.append(h)
    aspect_ratios.append(w / h)

widths = np.array(widths)
heights = np.array(heights)
aspect_ratios = np.array(aspect_ratios)

print("Ancho: mean={:.1f}, std={:.1f}, min={}, max={}".format(widths.mean(), widths.std(), widths.min(), widths.max()))
print("Alto: mean={:.1f}, std={:.1f}, min={}, max={}".format(heights.mean(), heights.std(), heights.min(), heights.max()))
print("Aspect ratio: mean={:.2f}, std={:.2f}, min={:.2f}, max={:.2f}".format(aspect_ratios.mean(), aspect_ratios.std(), aspect_ratios.min(), aspect_ratios.max()))

plt.figure(figsize=(12,3))

plt.subplot(1,3,1)
plt.hist(widths, bins=30, color='skyblue')
plt.title("Distribución de ancho")

plt.subplot(1,3,2)
plt.hist(heights, bins=30, color='salmon')
plt.title("Distribución de alto")

plt.subplot(1,3,3)
plt.hist(aspect_ratios, bins=30, color='lightgreen')
plt.title("Distribución de aspect ratio")

plt.show()


Todas las imágenes del dataset han sido redimensionadas a un tamaño uniforme de 160x160 píxeles, con un aspect ratio de 1.0. Esto garantiza que el modelo reciba entradas homogéneas, facilitando el procesamiento por lotes y evitando distorsiones o sesgos relacionados con diferentes tamaños o proporciones. Esta uniformidad es fundamental para asegurar un entrenamiento estable y eficiente, y permite comparar resultados de manera justa entre diferentes experimentos y arquitecturas.

## Media y Desviación Estándar de canales de color

In [ ]:
means = []
stds = []

#iteramos por batchss para no sarturar memoria
loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)

for imgs, _ in loader:
    means.append(imgs.mean(dim=[0,2,3]))  # mean por canal
    stds.append(imgs.std(dim=[0,2,3]))    # std por canal

mean = torch.stack(means).mean(dim=0)
std = torch.stack(stds).mean(dim=0)

print("Media por canal RGB:", mean)
print("Desviación estándar por canal RGB:", std)

# Histograma de intensidad por canal
all_pixels = []
for imgs, _ in loader:
    all_pixels.append(imgs)

all_pixels = torch.cat(all_pixels, dim=0)  # shape [N, C, H, W]

plt.figure(figsize=(8,4))
colors = ['r','g','b']
for i, color in enumerate(colors):
    plt.hist(all_pixels[:,i,:,:].numpy().flatten(), bins=50, color=color, alpha=0.5, label=f'Canal {color.upper()}')
plt.title("Histograma de intensidad por canal RGB")
plt.xlabel("Valor de píxel")
plt.ylabel("Cantidad")
plt.legend()
plt.show()


## Calidads de las imágenes

In [ ]:
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# --- Definición de Funciones de Puntuación ---
def blur_score(img):
    """Calcula el score de borrosidad usando la Varianza del Laplaciano."""
    img_gray = np.array(img.convert('L'))
    return cv2.Laplacian(img_gray, cv2.CV_64F).var()

def brightness_score(img):
    """Calcula el brillo promedio (media del píxel) en escala de grises."""
    img_gray = np.array(img.convert('L'))
    return img_gray.mean()

def contrast_score(img):
    """Calcula el contraste (desviación estándar) en escala de grises."""
    img_gray = np.array(img.convert('L'))
    return img_gray.std()

# --- Umbrales (los que definiste) ---
blur_threshold = 10      # <10: borrosa
dark_threshold = 30      # <30: muy oscura
bright_threshold = 220   # >220: muy brillante
contrast_threshold = 15  # <15: poco contraste

# --- Lógica de Conteo Detallado ---
total_images = len(train_dataset)

# Contadores para cada tipo de problema (pueden superponerse)
count_blur = 0
count_dark = 0
count_bright = 0
count_low_contrast = 0
count_multiple_problems = 0
total_problem_count = 0

problem_images_examples = [] # Para guardar un máximo de 20 ejemplos para plotear

print(f"Iniciando el análisis de calidad detallado en {total_images} imágenes...")

# Iterar sobre todo el dataset de entrenamiento
for i, (img, label) in enumerate(train_dataset):
    # Asegurarse de tener una imagen PIL para los cálculos
    pil_img = img if isinstance(img, Image.Image) else transforms.ToPILImage()(img)
    
    blur = blur_score(pil_img)
    bright = brightness_score(pil_img)
    contrast = contrast_score(pil_img)
    
    current_problems = 0
    is_problematic = False
    
    # 1. Borrosidad
    if blur < blur_threshold:
        count_blur += 1
        current_problems += 1
        is_problematic = True

    # 2. Brillo (Oscuro)
    if bright < dark_threshold:
        count_dark += 1
        current_problems += 1
        is_problematic = True

    # 3. Brillo (Brillante)
    if bright > bright_threshold:
        count_bright += 1
        current_problems += 1
        is_problematic = True

    # 4. Contraste (Bajo)
    if contrast < contrast_threshold:
        count_low_contrast += 1
        current_problems += 1
        is_problematic = True

    # Contar si tiene MÚLTIPLES problemas (2 o más)
    if current_problems >= 2:
        count_multiple_problems += 1
        
    # Contar el total de imágenes con AL MENOS un problema
    if is_problematic:
        total_problem_count += 1
        # Guardar ejemplos solo si no hemos alcanzado el límite de 20
        if len(problem_images_examples) < 20:
            problem_images_examples.append((pil_img, label, blur, bright, contrast))


# --- Resultados del Conteo ---
def get_percentage(count, total):
    return (count / total) * 100

print("-" * 50)
print(f"Total de imágenes analizadas: {total_images}")
print("\n--- Desglose de Problemas (Superpuestos) ---")
print(f"1. Borrosidad (<{blur_threshold:.1f}):      {count_blur} ({get_percentage(count_blur, total_images):.2f}%)")
print(f"2. Muy Oscuras (<{dark_threshold:.1f}):    {count_dark} ({get_percentage(count_dark, total_images):.2f}%)")
print(f"3. Muy Brillantes (>{bright_threshold:.1f}): {count_bright} ({get_percentage(count_bright, total_images):.2f}%)")
print(f"4. Bajo Contraste (<{contrast_threshold:.1f}): {count_low_contrast} ({get_percentage(count_low_contrast, total_images):.2f}%)")

print("\n--- Resumen General ---")
print(f"Imágenes con MÚLTIPLES problemas (>=2): {count_multiple_problems} ({get_percentage(count_multiple_problems, total_images):.2f}%)")
print(f"Imágenes problemáticas TOTALES (>=1):    {total_problem_count} ({get_percentage(total_problem_count, total_images):.2f}%)")
print("-" * 50)

# --- Visualización de Ejemplos ---
# (El código de visualización de ejemplos sigue siendo el mismo)
if problem_images_examples:
    num_plots = len(problem_images_examples)
    rows = (num_plots + 4) // 5  # Calcular filas necesarias para 5 columnas
    
    fig, axes = plt.subplots(rows, 5, figsize=(10, 3 * rows))
    
    # Manejar el caso de una sola fila de plots
    axes_flat = axes.flatten() if isinstance(axes, np.ndarray) and axes.ndim > 1 else (axes.flatten() if isinstance(axes, np.ndarray) else [axes])
    
    for ax, (img, label, blur, bright, contrast) in zip(axes_flat, problem_images_examples):
        # Asumiendo que 'class_names_simple' está definido. Si no, usa f"Clase {label}"
        try:
            class_name = class_names_simple[label]
        except (NameError, TypeError):
            class_name = f"Clase {label}"

        ax.imshow(img)
        ax.set_title(f"{class_name}\nB: {blur:.1f}, Br: {bright:.1f}, C: {contrast:.1f}", fontsize=8)
        ax.axis('off')

    # Ocultar ejes sobrantes
    for i in range(num_plots, len(axes_flat)):
        axes_flat[i].axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron ejemplos problemáticos para plotear con los umbrales definidos.")

## Brillo, Contraste y Saturación

In [ ]:
import colorsys

brightness_list = []
contrast_list = []
saturation_list = []

for img, _ in train_dataset:
    # Convertir a PIL si no lo es
    pil_img = img if isinstance(img, Image.Image) else transforms.ToPILImage()(img)
    
    # Convertir a array
    img_arr = np.array(pil_img)
    
    # Brillo y contraste (en escala de grises)
    gray = cv2.cvtColor(img_arr, cv2.COLOR_RGB2GRAY)
    brightness_list.append(gray.mean())
    contrast_list.append(gray.std())
    
    # Saturación promedio
    hsv = cv2.cvtColor(img_arr, cv2.COLOR_RGB2HSV)
    saturation_list.append(hsv[:,:,1].mean())

# Convertir a numpy
brightness_list = np.array(brightness_list)
contrast_list = np.array(contrast_list)
saturation_list = np.array(saturation_list)

# Histogramas
plt.figure(figsize=(10,4))

plt.subplot(1,3,1)
plt.hist(brightness_list, bins=50, color='gold', alpha=0.7)
plt.title("Distribución de brillo")
plt.xlabel("Valor medio de píxel")
plt.ylabel("Cantidad de imágenes")

plt.subplot(1,3,2)
plt.hist(contrast_list, bins=50, color='lightblue', alpha=0.7)
plt.title("Distribución de contraste")
plt.xlabel("Desviación estándar de píxeles")
plt.ylabel("Cantidad de imágenes")

plt.subplot(1,3,3)
plt.hist(saturation_list, bins=50, color='lightgreen', alpha=0.7)
plt.title("Distribución de saturación")
plt.xlabel("Valor medio de saturación")
plt.ylabel("Cantidad de imágenes")

plt.tight_layout()
plt.show()


## Análisis del espectro de frecuencia (FFT)

In [ ]:
import numpy as np
import cv2
from PIL import Image

def calcular_espectro_fft(img):
    """
    Calcula el espectro de magnitud de Fourier centrado de una imagen RGB/PIL/torch.Tensor.
    """
    # Convertir a PIL si es tensor
    if isinstance(img, torch.Tensor):
        img = img.numpy().transpose(1, 2, 0)  # C,H,W -> H,W,C
        img = (img * 255).astype(np.uint8)    # De [0,1] a [0,255]
    elif isinstance(img, Image.Image):
        img = np.array(img)
    
    # Convertir a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # FFT 2D y centrar frecuencia baja
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    
    # Magnitud logarítmica
    magnitude = 20 * np.log(np.abs(fshift) + 1e-8)  # epsilon para evitar log(0)
    return magnitude


In [ ]:
from tqdm.notebook import tqdm 
import random

num_samples = min(9000, len(train_dataset))  # limitar muestra por velocidad
indices_muestra = random.sample(range(len(train_dataset)), num_samples)

# Inicializar con la primera imagen
espectro_acumulado = np.zeros_like(calcular_espectro_fft(train_dataset[0][0]))

print(f"Calculando espectro promedio sobre {num_samples} imágenes...")
for i in tqdm(indices_muestra):
    img, _ = train_dataset[i]
    espectro = calcular_espectro_fft(img)
    if espectro.shape == espectro_acumulado.shape:
        espectro_acumulado += espectro

espectro_promedio = espectro_acumulado / num_samples


In [ ]:
import matplotlib.pyplot as plt

espectro_norm = (espectro_promedio - np.min(espectro_promedio)) / (np.max(espectro_promedio) - np.min(espectro_promedio))

plt.figure(figsize=(6, 6))
plt.imshow(espectro_norm, cmap='gray')
plt.title(f'Espectro de Magnitud Promedio (N={num_samples})')
plt.colorbar(label='Intensidad Normalizada (Log)')
plt.axis('off')
plt.show()


## Estimación de ruido local

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from tqdm.notebook import tqdm
# Asumo que 'transforms' viene de PyTorch si usas un dataset estándar
try:
    import torchvision.transforms as transforms
except ImportError:
    # Definir un dummy si no se usa PyTorch (solo si no se requiere la conversión)
    class DummyTransform:
        def __call__(self, img):
            return img
    transforms = type('transforms', (object,), {'ToPILImage': lambda: DummyTransform()})()

# Celda 1: Cálculo de Ruido y Almacenamiento de Índices

# Parámetros para la estimación:
PATCH_SIZE = 10 
num_muestras_ruido = min(9000, len(train_dataset)) 

def estimar_ruido_local(image_rgb):
    """Estima el ruido como la desviación estándar local en la imagen."""
    
    # Manejo del formato de la imagen para cv2
    if image_rgb.dtype != np.uint8:
        # Asume que si no es uint8, está normalizada a 0-1, la escalamos.
        # Ajusta esta línea si sabes que tus arrays NumPy tienen otro rango (ej. 0-255 en float)
        image_rgb = (image_rgb * 255).astype(np.uint8) 
        
    gray_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    H, W = gray_image.shape
    
    if H < PATCH_SIZE or W < PATCH_SIZE:
         return 0 # Si es muy pequeña, la omitimos o asignamos un valor bajo
         
    varianzas_locales = []
    
    # Tomar 10 parches aleatorios en la imagen para estimar la variación local
    for _ in range(10): 
        x = np.random.randint(0, W - PATCH_SIZE)
        y = np.random.randint(0, H - PATCH_SIZE)
        
        patch = gray_image[y:y + PATCH_SIZE, x:x + PATCH_SIZE]
        varianzas_locales.append(np.std(patch))
        
    return np.mean(varianzas_locales)

# Almacenamos tuplas (índice, ruido_estimado)
ruidos_estimados_con_indice = []
print(f"Estimando ruido local en {num_muestras_ruido} imágenes...")

for i in tqdm(range(num_muestras_ruido)):
    # Asume que train_dataset[i] devuelve (imagen, etiqueta)
    img, _ = train_dataset[i] 
    
    # Manejo de formatos: si es un objeto Tensor/PIL, lo convertimos a NumPy array
    if not isinstance(img, np.ndarray):
        try:
            # Si es un Tensor de PyTorch, lo convertimos a NumPy (CHW -> HWC)
            # Nota: Esto asume el orden de canales de PyTorch
            img = np.array(transforms.ToPILImage()(img))
        except:
             # Si falla la conversión a PIL/NumPy (caso extremo), usamos el array directamente si es posible
             if hasattr(img, 'numpy'):
                 img = img.numpy().transpose((1, 2, 0))
             else:
                 img = np.array(img)
    
    ruido_est = estimar_ruido_local(img)
    ruidos_estimados_con_indice.append((i, ruido_est))

# Convertir a array NumPy para cálculos estadísticos
ruidos_array = np.array([r[1] for r in ruidos_estimados_con_indice])
media_ruido = np.mean(ruidos_array)
std_ruido = np.std(ruidos_array)

print("\n--- Resultados Estadísticos ---")
print(f"Media ($\mu$): {media_ruido:.2f}, Desviación Estándar ($\sigma$): {std_ruido:.2f}")

# --- Identificación de Outliers (Para la visualización) ---
umbral_alto = media_ruido + 2 * std_ruido
indices_ruido_alto = [
    i for i, ruido in ruidos_estimados_con_indice if ruido > umbral_alto
]

print(f"Umbral para ruido alto ($\mu + 2\sigma$): {umbral_alto:.2f}")
print(f"Se encontraron {len(indices_ruido_alto)} imágenes con $\sigma_n$ superior a este umbral.")

In [ ]:

plt.figure(figsize=(6, 4))
plt.hist(ruidos_array, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(media_ruido, color='red', linestyle='dashed', linewidth=1, label=f'Media: {media_ruido:.2f}')
plt.title('Distribución de la Estimación de Ruido Local ($\sigma_n$)')
plt.xlabel('Desviación Estándar Local (Estimación $\sigma_n$)')
plt.ylabel('Frecuencia')
plt.legend()
plt.grid(axis='y', alpha=0.5)
plt.show()

print("El alto valor de la media (23.48) indica una alta granularidad/textura en el dataset.")

In [ ]:
# Celda 3: Visualización de Ejemplos de Ruido Alto (Outliers)

num_a_mostrar = min(10, len(indices_ruido_alto))
indices_a_mostrar = indices_ruido_alto[:num_a_mostrar]

if num_a_mostrar > 0:
    plt.figure(figsize=(10, 5))
    plt.suptitle(f"Ejemplos de Imágenes con Ruido Local Alto ($\sigma_n > {umbral_alto:.2f}$)", fontsize=16)

    for i, idx_original in enumerate(indices_a_mostrar):
        img, label_idx = train_dataset[idx_original]
        
        # Volvemos a convertir a NumPy si es necesario
        if not isinstance(img, np.ndarray):
            try:
                img = np.array(transforms.ToPILImage()(img))
            except:
                 if hasattr(img, 'numpy'):
                     img = img.numpy().transpose((1, 2, 0))
                 else:
                     img = np.array(img)

        # Buscar el valor de ruido específico
        ruido_especifico = [r[1] for r in ruidos_estimados_con_indice if r[0] == idx_original][0]
        
        plt.subplot(2, 5, i + 1)
        plt.imshow(img) 
        plt.title(f"Clase: {label_idx}\n$\\sigma_n$: {ruido_especifico:.2f}", fontsize=10)
        plt.axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
else:
    print("No hay suficientes imágenes con ruido extremadamente alto (o el umbral es muy estricto) para mostrar.")

# 4. Principales Insights del EDA


## 1. Geometría y Distribución (Puntos 1 y 2)

### Hallazgos Clave
* **Balance de Clases:** El *dataset* está **perfectamente balanceado** (aprox. 950 imágenes por clase), sin ninguna clase minoritaria significativa.
* **Uniformidad Geómetrica:** Todas las imágenes son de **$160 \times 160$ píxeles** con una relación de aspecto cuadrada ($1.0$).

### Decisiones de Preprocesamiento
| Decisión | Justificación del EDA |
| :--- | :--- |
| **No se requiere *weighted loss*** | El balance perfecto de clases elimina la necesidad de técnicas de mitigación de desbalance (ej. SMOTE, pérdida ponderada). La **precisión** global es una métrica confiable. |
| **Transformación de *Resize*** | Ya que todas las imágenes son uniformes ($160 \times 160$), el *resize* no es un paso de corrección, sino de **adaptación** si se usa una arquitectura pre-entrenada (ej. a $224 \times 224$ o $256 \times 256$). |
| **Random Resized Crop (RRC)** | Obligatorio, dado que la variación de escala es $0$. El RRC es esencial para introducir **variabilidad de tamaño** y **eliminar los posibles bordes no informativos** (picos en $0.0/1.0$ del histograma). |


## 2. Color y Brillo (Puntos 3, 4 y 5)

### Hallazgos Clave
* **Iluminación/Varianza:** La media RGB general ($\mu \approx 0.45$) indica una ligera oscuridad, pero la desviación estándar ($\sigma \approx 0.27$) revela una **alta varianza de contraste y brillo**.
* **Picos Extremos:** El histograma RGB muestra picos fuertes en los valores $0.0$ y $1.0$ (especialmente en el canal Azul), indicando la presencia de **fondos negros/blancos y luces/sombras extremas**.
* **Baja Saturación:** La distribución de saturación es **asimétrica**, concentrándose en valores bajos ($\approx 50$), lo que implica que la mayoría de las imágenes tienen colores apagados o moderados.
* **Calidad:** Solo el **$1.73\%$** de las imágenes son problemáticas (extremos de brillo/contraste), por lo que **no se requiere limpieza manual**.

### Decisiones de Preprocesamiento y Augmentation
| Decisión | Justificación del EDA |
| :--- | :--- |
| **Normalización (RGB)** | Necesaria para centrar los datos y optimizar la convergencia del modelo. Se aplicará con los valores calculados: $\mu=[0.4625, 0.4580, 0.4298]$ y $\sigma=[0.2748, 0.2690, 0.2856]$. |
| **Augmentation de Saturación** | **Prioridad Alta.** La baja saturación del *dataset* exige una **variación agresiva** (ej. rango $\mathbf{(0.5, 1.5)}$) para evitar que el modelo se sobreajuste a los colores apagados. |
| **Augmentation de Brillo/Contraste** | **Prioridad Media.** Necesario para mitigar el $1.73\%$ de *outliers* extremos y manejar la alta varianza natural del contraste (ej. rango $\mathbf{(0.7, 1.3)}$). |


## 3. Textura y Frecuencia (Puntos 6 y 7)

### Hallazgos Clave
* **FFT (Global):** La mayor energía se concentra en las **bajas frecuencias** (formas grandes, fondos uniformes), con líneas axiales que sugieren estructuras fuertes (horizontales/verticales) o artefactos de *resizing*.
* **Ruido Local ($\sigma_n$):** La media de la desviación estándar local es **alta** ($\mu \approx 23.4$), lo que indica una **rica y alta granularidad/textura** a nivel de parche.
* **Outliers Texturales:** Se identificó un pequeño grupo de imágenes (aprox. $280$) con textura excepcionalmente alta ($\sigma_n > 42.4$).

### Decisiones de Modelado y Augmentation
| Decisión | Justificación del EDA |
| :--- | :--- |
| **Augmentation de Ruido** | **Esencial.** Aplicar **Ruido Gaussiano aleatorio** con baja probabilidad, imitando el pequeño grupo de *outliers* de alta $\sigma_n$, haciendo el modelo robusto a variaciones texturales y ruido ausente en el FFT. |
| **Augmentation Geométrico** | **Necesario.** Incluir transformaciones como **rotaciones leves** o **shear** para forzar al modelo a centrarse en la **forma** del objeto (baja frecuencia) independientemente de su orientación, rompiendo la uniformidad axial del FFT. |
| **Elección de Arquitectura** | La alta granularidad ($\sigma_n$) sugiere que las arquitecturas basadas en convolución (ej. **ResNet, VGG**) serán efectivas, ya que sus filtros pequeños ($3 \times 3$) están bien equipados para capturar la rica información textural local. |

# 5. Separación de Datos (Train/Val/Test)

1. **Train dataset original**  
   - Contiene todas las imágenes de entrenamiento (`split="train"`).  
   - Se usa para crear un **split interno** de entrenamiento y validación.

2. **Split interno (train / val)**  
   - 80% de las imágenes → subset de entrenamiento (`train_subset`)  
   - 20% de las imágenes → subset de validación (`val_subset`)  
   - Permite monitorear **loss y accuracy** durante el entrenamiento sin tocar el test final.

3. **Test final (`val_dataset`)**  
   - El split `val` de Imagenette se mantiene intacto y se usa como **test final**.  
   - Garantiza que la evaluación final sea objetiva e independiente de los datos de entrenamiento.

In [ ]:
# Valores calculados en el EDA de Imagenette160
imagenette_mean = [0.467, 0.448, 0.397]
imagenette_std  = [0.230, 0.226, 0.228]

normalize = transforms.Normalize(mean=imagenette_mean, std=imagenette_std)

train_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.RandomResizedCrop(160, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),         
    transforms.RandomRotation(degrees=10),
    
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.5),
    

    transforms.RandomApply([transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0))], p=0.5),
    
    transforms.ToTensor(),
    normalize
])

# 3. Transformaciones de Validación (sin aleatoriedad)
val_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    normalize
])

test_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    normalize

])

In [ ]:
from torch.utils.data import random_split

val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_subset.dataset.transform = train_transform
val_subset.dataset.transform = val_transform

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader_internal = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

test_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader.dataset.transform = test_transform

# 6. Funciones Train y Eval Genéricas

In [ ]:
def train_model(model, train_loader, val_loader, config, device, patience=5):
    """
    Entrena un modelo con early stopping, logging en wandb y retorno del historial + mejor modelo.
    
    Args:
        model: nn.Module, modelo a entrenar
        train_loader: DataLoader del conjunto de entrenamiento
        val_loader: DataLoader del conjunto de validación
        config: dict con hiperparámetros (lr, optimizer, epochs, etc.)
        device: "cuda" o "cpu"
        patience: int, cantidad de epochs sin mejora para early stopping

    Returns:
        best_model: modelo con menor val_loss
        history: diccionario con listas de train/val loss y accuracy
    """
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    if config["optimizer"] == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    elif config["optimizer"] == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
    else:
        raise ValueError(f"Optimizer {config['optimizer']} not supported")

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    best_val_loss = float("inf")
    best_model_state = None
    patience_counter = 0

    for epoch in range(config["epochs"]):
        # ---- Entrenamiento ----
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # ---- Validación ----
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = outputs.max(1)
                correct_val += preds.eq(labels).sum().item()
                total_val += labels.size(0)

        val_loss /= total_val
        val_acc = correct_val / total_val

        # ---- Guardar en historial ---
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        # ---- Log en W&B ----
        wandb.log({
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "epoch": epoch
        })

        print(f"[{epoch+1}/{config['epochs']}] "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        # ---- Early stopping y guardar mejor modelo ----
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()  # guarda pesos
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"⏹ Early stopping en epoch {epoch+1}")
                break

    # ---- Devolver mejor modelo ----
    best_model = model
    if best_model_state is not None:
        best_model.load_state_dict(best_model_state)

    return best_model, history


In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    return {"loss": avg_loss, "accuracy": accuracy}


# 7. Modelo Baseline

In [ ]:
class CNNBaseline(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNBaseline, self).__init__()
        
        self.features = nn.Sequential(
            # Bloque 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.1),
            
            # Bloque 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            
            # Bloque 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )
        
        self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(128 * 20 * 20, 128),  # De 256 a 128
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),                
        nn.Linear(128, num_classes)
    )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

from torchinfo import summary
model = CNNBaseline(num_classes=10).to(device)
summary(model, input_size=(1, 3, 160, 160))  


In [ ]:
sweep_config = {
    "method": "bayes",  
    "name": "cnn_baseline_sweep",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-2, 1e-3, 1e-4,]},
        "batch_size": {"values": [64, 128]},
        "optimizer": {"values": ["Adam", "SGD"]},
        "dropout1": {"values": [0.1, 0.2]},
        "dropout2": {"values": [0.2, 0.3, 0.4]},
        "dropout3": {"values": [0.3, 0.4, 0.5]},
        "epochs": {"value": 15}  # fija 15 epochs, early stopping lo limitará
    }
}

In [ ]:
def sweep_train():
    # Config de W&B
    wandb.init(
        project="imagenette",
        config=wandb.config,
        settings=wandb.Settings(console="off")  
    )    
    config = wandb.config
    
    # Ajustamos el modelo con los dropout del sweep
    class CNNBaselineSweep(nn.Module):
        def __init__(self, num_classes=10):
            super().__init__()
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, 3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Dropout(config.dropout1),
                
                nn.Conv2d(32, 64, 3, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Dropout(config.dropout2),
                
                nn.Conv2d(64, 128, 3, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Dropout(config.dropout3)
            )
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(128*20*20, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(config.dropout3),
                nn.Linear(128, num_classes)
            )
        def forward(self, x):
            x = self.features(x)
            x = self.classifier(x)
            return x

    model = CNNBaselineSweep(num_classes=10).to(device)
    
    train_loader_sweep = DataLoader(train_subset, batch_size=config.batch_size, shuffle=True, num_workers=NUM_WORKERS)
    val_loader_sweep   = DataLoader(val_subset, batch_size=config.batch_size, shuffle=False, num_workers=NUM_WORKERS)

    
    # Entrenar
    best_model, history = train_model(model, train_loader_sweep, val_loader_sweep, config, device, patience=3)

    tmp_path = f"best_model_{wandb.run.id}.pth"
    torch.save(best_model.state_dict(), tmp_path)

    artifact = wandb.Artifact(
        name=f"cnn_baseline_sweep_{wandb.run.id}",
        type="model",
        description=f"Mejor modelo CNNBaseline del sweep con lr={config.lr}, dropout1={config.dropout1}, dropout2={config.dropout2}, dropout3={config.dropout3}, batch_size={config.batch_size}"
    )
    artifact.add_file(tmp_path)
    wandb.log_artifact(artifact)

    wandb.finish()  # opcional

    return best_model, history



In [ ]:
#Esto genera muchos prints! por lo que el output fue borrado. 
sweep_id = wandb.sweep(sweep_config, project="imagenette")
wandb.agent(sweep_id, function=sweep_train, count=25)  # count = número de combinaciones a probar


In [ ]:
# Recuperar la mejor run del sweep (la que tuvo mejor val_acc)
entity = "kidnixt-ort"     
project = "imagenette"      
sweep_id = "yt8hz9zs"  

api = wandb.Api()
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

# Esto devuelve la mejor run según la métrica objetivo del sweep
best_run = sweep.best_run()

print(f"🏆 Mejor run: {best_run.name} ({best_run.id})")
print("Config:", best_run.config)
print("Summary:", dict(best_run.summary))


In [ ]:
#entrenamos con esos hiperparámetros, luego esto se obtendra como un artifact de wandb.
class CNNBaselineBestParams(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNBaselineBestParams, self).__init__()
        
        self.features = nn.Sequential(
            # Bloque 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            
            # Bloque 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            
            # Bloque 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.4)
        )
        
        self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(128 * 20 * 20, 128),  # De 256 a 128
        nn.ReLU(inplace=True),
        nn.Dropout(0.4),                
        nn.Linear(128, num_classes)
    )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

from torchinfo import summary
model = CNNBaselineBestParams(num_classes=10).to(device)
summary(model, input_size=(1, 3, 160, 160))  


In [ ]:
BATCH_SIZE = best_run.config['batch_size']
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader_internal = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

LR = best_run.config['lr']
OPTIMIZER = best_run.config['optimizer']
EPOCHS = best_run.config['epochs']
config = {
    "lr": LR,
    "optimizer": OPTIMIZER,
    "epochs": EPOCHS
}

run_name = "baseline-bestparams using " + best_run.name
tags = ["baseline", "bestparams"]  # puedes usar ["baseline"] para el otro caso

wandb.init(
    project="imagenette",
    name=run_name,
    tags=tags,
    config=config  # puedes incluir lr, optimizer, etc.
)

# Re-inicializamos el modelo
model = CNNBaselineBestParams(num_classes=10).to(device)
best_model, history = train_model(model, train_loader, val_loader_internal, config, device, patience=3)

wandb.finish()


In [ ]:

# Número de epochs
epochs = range(1, len(history["train_loss"]) + 1)

# ---- Loss ----
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(epochs, history["train_loss"], label="Train Loss")
plt.plot(epochs, history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid(True)

# ---- Accuracy ----
plt.subplot(1,2,2)
plt.plot(epochs, history["train_acc"], label="Train Acc")
plt.plot(epochs, history["val_acc"], label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
results_test = evaluate(best_model, test_loader, device)

print("📊 Resultados en Test:")
print(f"Loss: {results_test['loss']:.4f}")
print(f"Accuracy: {results_test['accuracy']:.4f}")


In [ ]:
sweep_config = {
    "method": "bayes",  
    "name": "cnn_baseline_TEST",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-2, 1e-3, 1e-4,]},
        "batch_size": {"values": [64, 128]},
        "optimizer": {"values": ["Adam", "SGD"]},
        "dropout1": {"values": [0.1, 0.2]},
        "dropout2": {"values": [0.2, 0.3, 0.4]},
        "dropout3": {"values": [0.3, 0.4, 0.5]},
        "epochs": {"value": 15}  # fija 15 epochs, early stopping lo limitará
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="imagenette")
wandb.agent(sweep_id, function=sweep_train, count=2)  # count = número de combinaciones a probar

In [ ]:
import wandb
import os

file_path = "pyproject.toml"

# Asegurarnos que no haya una sesión previa viva
wandb.finish()

print(f"✅ Existe el archivo?: {os.path.exists(file_path)}")
print(f"📏 Tamaño: {os.path.getsize(file_path) / 1_000_000:.2f} MB")

# Nuevo run completamente independiente
run = wandb.init(project="imagenette", job_type="manual-upload", reinit=True)

artifact = wandb.Artifact(
    name="keje",
    type="dataset",
    description="Subida manual de modelo preentrenado"
)

artifact.add_file(file_path, name="best_model_manual.pth")
print("📎 Archivo agregado al artifact.")

artifact.save()
wandb.log_artifact(artifact)


run.finish()
print("✅ Finalizado.")


In [ ]:
import random
import wandb
import os
from pathlib import Path
import time # Importar time, aunque la espera principal es con artifact.wait()

def model(training_data: float) -> float:
    """Simulación de modelo."""
    return training_data * 2 + random.uniform(-1, 1)

# Simulación de pesos y ruido
weights = random.random()
noise = random.random() / 5

# Hiperparámetros y configuración
config = {
    "epochs": 10,
    "learning_rate": 0.01,
}

# Definir la ruta local del archivo del modelo (usando Path para robustez en Windows)
PATH = Path("model.txt") 

# 1. Limpieza: Asegúrate de que no haya un archivo anterior que cause conflictos.
if PATH.exists():
    os.remove(PATH)
    
print(f"El archivo del modelo se guardará en: {PATH.resolve()}")

# Usar el manejador de contexto para inicializar y cerrar W&B runs
with wandb.init(project="imagenette", entity="kidnixt-ort", config=config) as run:
    
    # Simulación de entrenamiento
    for epoch in range(config["epochs"]):
        xb = weights + noise
        yb = weights + noise * 2

        y_pred = model(xb)
        loss = (yb - y_pred) ** 2

        print(f"epoch={epoch}, loss={y_pred}")
        run.log({"epoch": epoch, "loss": loss})

    # 2. Guarda el modelo localmente
    # Usamos str(PATH) para asegurarnos de que el método open() funcione correctamente
    with open(str(PATH), "w") as f:
        f.write(str(weights)) 

    # 3. Define y añade el Artifact
    model_artifact_name = "model-demo"
    artifact = wandb.Artifact(
        name=model_artifact_name, 
        type="model", 
        description="My trained model"
    )
    # Añadir el archivo local al Artifact
    artifact.add_file(local_path=str(PATH))
    
    # 4. REGISTRA el Artifact con el run (Inicia la subida asíncrona)
    run.log_artifact(artifact)
    
    # 5. ¡Paso CLAVE para Windows! Esperar a que la subida termine.
    print("\nEsperando a que los artifacts terminen de subir antes de finalizar el run...")
    try:
        # Esto bloquea la ejecución hasta que la subida del artifact finaliza.
        artifact.wait() 
        print("Subida de artifacts finalizada exitosamente.")
    except ValueError as e:
        print(f"\n--- ERROR CRÍTICO DE SUBIDA ---")
        print(f"W&B falló al subir el Artifact (ValorError): {e}")
        print("Esto sugiere un problema de permisos o bloqueo de disco.")
        print("Intenta ejecutar el terminal/IDE como Administrador.")

print(f"\nProceso finalizado. Comprueba el Artifact en tu dashboard de W&B.")

# Limpieza opcional: si quieres borrar el archivo local después de la subida
# os.remove(PATH)

In [ ]:
# Artifact name specifies the specific artifact version within our team's project

TEAM_ENTITY = "kidnixt-ort"  # Your W&B team or username
PROJECT = "imagenette"       # Your W&B project name
artifact_name = f'{TEAM_ENTITY}/{PROJECT}/{model_artifact_name}'
print("Artifact name: ", artifact_name)

REGISTRY_NAME = "Model" # Name of the registry in W&B
COLLECTION_NAME = "DemoModels"  # Name of the collection in the registry

# Create a target path for our artifact in the registry
target_path = f"wandb-registry-{REGISTRY_NAME}/{COLLECTION_NAME}"
print("Target path: ", target_path)

run = wandb.init(entity=TEAM_ENTITY, project=PROJECT)
model_artifact = run.use_artifact(artifact_or_name=artifact_name, type="model")
run.link_artifact(artifact=model_artifact, target_path=target_path)
run.finish()


In [ ]:
import os
print(os.path.exists("best_model_9mrxi4y8.pth"))


In [1]:
import os
from pathlib import Path
import wandb

# Imprime las variables de entorno que controlan las rutas
print("--- VARIABLES DE ENTORNO DE W&B ---")
print(f"WANDB_CACHE_DIR (Caché global): {os.getenv('WANDB_CACHE_DIR', 'No definida')}")
print(f"WANDB_DATA_DIR (Archivos temporales): {os.getenv('WANDB_DATA_DIR', 'No definida')}")
print(f"WANDB_CONFIG_DIR (Configuración/Login): {os.getenv('WANDB_CONFIG_DIR', 'No definida')}")
print("-" * 35)

# Si tienes un run activo, esta función te diría dónde están los logs y archivos de ese run específico.
# Puedes ver dónde se guardarían los logs si inicias un run:
try:
    import wandb
    with wandb.init(project="check_paths", mode="dryrun") as run: # dryrun previene la subida real
        print("--- RUTAS DE UN RUN DE W&B (DRY RUN) ---")
        # El directorio donde se guardan los archivos de LOG/RUN (por defecto, en el directorio del script)
        print(f"Directorio de logs del Run (WANDB_DIR): {Path(run.dir).resolve()}")
        # El directorio de caché de Artifacts usado:
        print(f"Directorio de caché de Artifacts: {Path(wandb.util.get_cache_dir()).resolve()}")
        print("-" * 35)
except Exception as e:
    print(f"No se pudo inicializar W&B para verificar rutas: {e}")

--- VARIABLES DE ENTORNO DE W&B ---
WANDB_CACHE_DIR (Caché global): No definida
WANDB_DATA_DIR (Archivos temporales): No definida
WANDB_CONFIG_DIR (Configuración/Login): No definida
-----------------------------------


--- RUTAS DE UN RUN DE W&B (DRY RUN) ---
Directorio de logs del Run (WANDB_DIR): C:\Users\kidni\Desktop\imagenette\wandb\offline-run-20250930_194004-4wno5jec\files


Traceback (most recent call last):
  File "C:\Users\kidni\AppData\Local\Temp\ipykernel_17976\146756198.py", line 21, in <module>
    print(f"Directorio de caché de Artifacts: {Path(wandb.util.get_cache_dir()).resolve()}")
AttributeError: module 'wandb.util' has no attribute 'get_cache_dir'


No se pudo inicializar W&B para verificar rutas: module 'wandb.util' has no attribute 'get_cache_dir'


In [2]:
import wandb
print("wandb version:", wandb.__version__)
print("wandb path:", wandb.__file__)

wandb version: 0.22.1
wandb path: c:\Users\kidni\Desktop\imagenette\.venv\lib\site-packages\wandb\__init__.py


In [4]:
import random 
import wandb
import os
from pathlib import Path
import time

# =======================================================
# PASO FINAL Y CLAVE: FORZAR LA VARIABLE EN EL ENTORNO DE PYTHON
# =======================================================
# 1. Creamos y forzamos la variable WANDB_DATA_DIR. 
# Esto obliga a W&B a usar C:\wandb_temp en lugar de la ruta bloqueada de AppData.
# ¡Asegúrate de que la carpeta C:\wandb_temp exista!
os.environ["WANDB_DATA_DIR"] = "C:\\wandb_temp" 
print(f"DEBUG: WANDB_DATA_DIR ha sido forzada a: {os.environ['WANDB_DATA_DIR']}")
# =======================================================

def model(training_data: float) -> float:
    """Model simulation for demonstration purposes."""
    return training_data * 2 + random.uniform(-1, 1) 

# Simulación de pesos y ruido
weights = random.random()
noise = random.random() / 5

# Hyperparameters and configuration
config = {
    "epochs": 10,
    "learning_rate": 0.01,
}

# 2. Ruta local del modelo
PATH = Path("model.txt") 
if PATH.exists():
    os.remove(PATH)

# Usar contexto manager para W&B
with wandb.init(project="imagenette", entity="kidnixt-ort", config=config) as run:
    
    # Simulación de entrenamiento
    for epoch in range(config["epochs"]):
        xb = weights + noise
        yb = weights + noise * 2

        y_pred = model(xb) 
        loss = (yb - y_pred) ** 2

        print(f"epoch={epoch}, loss={loss:.4f}")
        run.log({"epoch": epoch, "loss": loss})

    # 3. Guarda el modelo localmente
    with open(str(PATH), "w") as f:
        f.write(str(weights)) 

    # 4. Define y añade el Artifact
    model_artifact_name = "model-demo"
    artifact = wandb.Artifact(
        name=model_artifact_name, 
        type="model", 
        description="My trained model"
    )
    artifact.add_file(local_path=str(PATH))
    
    # 5. REGISTRA el Artifact
    run.log_artifact(artifact)
    
    # 6. Espera la subida (Mantenemos el try/except)
    print("\nEsperando a que el Artifact termine de subir...")
    try:
        artifact.wait() 
        print("Subida de Artifact finalizada exitosamente.")
    except ValueError as e:
        # Si esto falla ahora, es un bloqueo del sistema de archivos extremadamente raro, 
        # pero al menos la ruta de staging será la correcta.
        print(f"\n--- ERROR PERSISTENTE DE SUBIDA ---")
        print(f"W&B falló al subir Artifact (ValorError): {e}")
        print("Esto sugiere un bloqueo de disco/antivirus. Por favor, intenta la Solución Offline.")
        
print(f"\nProceso finalizado. Verifica el Artifact en W&B.")

DEBUG: WANDB_DATA_DIR ha sido forzada a: C:\wandb_temp


epoch=0, loss=0.4514
epoch=1, loss=0.0759
epoch=2, loss=0.0019
epoch=3, loss=0.4999
epoch=4, loss=0.1806
epoch=5, loss=0.6132
epoch=6, loss=0.4569
epoch=7, loss=0.0495
epoch=8, loss=0.0341
epoch=9, loss=0.0121

Esperando a que el Artifact termine de subir...
Subida de Artifact finalizada exitosamente.


epoch,▁▂▃▃▄▅▆▆▇█
loss,▆▂▁▇▃█▆▂▁▁
epoch,9
loss,0.01213



Proceso finalizado. Verifica el Artifact en W&B.
